In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
client_id = '################################'
client_secret = '################################'

credentials = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
spotify = spotipy.Spotify(client_credentials_manager = credentials)

In [15]:
name_artist = []
id_artist = []
name_song = []
id_song = []
popularity_song = []
year_song = []

for year in ['2021', '2022', '2023']:
    for i in range(0,1000,50):
        track_results = spotify.search(q='year:' + str(year), type='track', market = 'PE', limit=50, offset=i)
        for i, dic in enumerate(track_results['tracks']['items']):
            name_artist.append(dic['artists'][0]['name'])
            id_artist.append(dic['artists'][0]['id'])
            name_song.append(dic['name'])
            id_song.append(dic['id'])
            popularity_song.append(dic['popularity'])
            year_song.append(year)

In [16]:
df = pd.DataFrame({'name_artist':name_artist, 'name_song':name_song, 'id_song':id_song, 'popularity_song':popularity_song, 'id_artist':id_artist, 'year':year_song})
print(df.shape)
df.head()

(3000, 6)


,name_artist,name_song,id_song,popularity_song,id_artist,year
0,Yng Lvcas,La Bebe,6odrHMSLjP3aOXC4ipKUC6,86,1NNRWkhwmcXRimFYSBpB1y,2021
1,Rvssian,Nostálgico,2BEcap3xJtmiPaHKyfQi4e,80,1fctva4kpRbg2k3v7kwRuS,2021
2,Bad Bunny,Yonaguni,2JPLbjOn0wPCngEot2STUS,85,4q3ewBCX7sLwd24euuV69X,2021
3,Tainy,Lo Siento BB:/ (with Bad Bunny & Julieta Venegas),4gzsuuZypVbxs0Af1LSZyB,83,0GM7qgcRCORpGnfcN2tCiB,2021
4,Rauw Alejandro,Desesperados,6mmPpaltUZK7xjNlBPQQ0p,83,1mcTU81TzQhprhouKaTkpq,2021


In [18]:
popularity_artist = []
genres_artist = []
followers_artist = []

for id_a in df.id_artist:
    artist = spotify.artist(id_a)
    popularity_artist.append(artist['popularity'])
    genres_artist.append(artist['genres'])
    followers_artist.append(artist['followers']['total'])


In [19]:
df = df.assign(popularity_artist=popularity_artist, genres_artist=genres_artist, followers_artist=followers_artist)
df.head()

,name_artist,name_song,id_song,popularity_song,id_artist,year,popularity_artist,genres_artist,followers_artist
0,Yng Lvcas,La Bebe,6odrHMSLjP3aOXC4ipKUC6,86,1NNRWkhwmcXRimFYSBpB1y,2021,81,[],90161
1,Rvssian,Nostálgico,2BEcap3xJtmiPaHKyfQi4e,80,1fctva4kpRbg2k3v7kwRuS,2021,69,"[reggaeton, trap latino, urbano latino]",2387197
2,Bad Bunny,Yonaguni,2JPLbjOn0wPCngEot2STUS,85,4q3ewBCX7sLwd24euuV69X,2021,97,"[reggaeton, trap latino, urbano latino]",69786971
3,Tainy,Lo Siento BB:/ (with Bad Bunny & Julieta Venegas),4gzsuuZypVbxs0Af1LSZyB,83,0GM7qgcRCORpGnfcN2tCiB,2021,75,"[pop reggaeton, reggaeton, trap latino, urbano...",1358527
4,Rauw Alejandro,Desesperados,6mmPpaltUZK7xjNlBPQQ0p,83,1mcTU81TzQhprhouKaTkpq,2021,89,"[puerto rican pop, reggaeton, trap latino, urb...",16900141


In [20]:
audio_features = []
for id_s in df['id_song']:
  features = spotify.audio_features(id_s)
  audio_features.append(features)


In [21]:
df_features = pd.DataFrame(columns = list(audio_features[0][0].keys()))
df_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature


In [ ]:
null_dict = {'danceability': np.NaN, 'energy': np.NaN, 'key': np.NaN, 'loudness': np.NaN, 'mode': np.NaN, 'speechiness': np.NaN, 'acousticness': np.NaN, 'instrumentalness': np.NaN, 'liveness': np.NaN,
              'valence': np.NaN, 'tempo': np.NaN, 'type': np.NaN, 'id': np.NaN, 'uri': np.NaN, 'track_href': np.NaN, 'analysis_url': np.NaN, 'duration_ms': np.NaN, 'time_signature': np.NaN}

for features in audio_features:
    for feature in features:
        if feature is None:
            df_features = df_features.append(null_dict, ignore_index=True)
        else:
            df_features = df_features.append(feature, ignore_index=True)

print(df_features.shape)
df_features.head()

In [23]:
track_df = df.join(df_features)
print(track_df.shape)
track_df.head()

(3000, 27)


,name_artist,name_song,id_song,popularity_song,id_artist,year,popularity_artist,genres_artist,followers_artist,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Yng Lvcas,La Bebe,6odrHMSLjP3aOXC4ipKUC6,86,1NNRWkhwmcXRimFYSBpB1y,2021,81,[],90161,0.775,...,0.1140,0.707,169.781,audio_features,6odrHMSLjP3aOXC4ipKUC6,spotify:track:6odrHMSLjP3aOXC4ipKUC6,https://api.spotify.com/v1/tracks/6odrHMSLjP3a...,https://api.spotify.com/v1/audio-analysis/6odr...,223993,4
1,Rvssian,Nostálgico,2BEcap3xJtmiPaHKyfQi4e,80,1fctva4kpRbg2k3v7kwRuS,2021,69,"[reggaeton, trap latino, urbano latino]",2387197,0.734,...,0.2350,0.604,98.026,audio_features,2BEcap3xJtmiPaHKyfQi4e,spotify:track:2BEcap3xJtmiPaHKyfQi4e,https://api.spotify.com/v1/tracks/2BEcap3xJtmi...,https://api.spotify.com/v1/audio-analysis/2BEc...,209884,4
2,Bad Bunny,Yonaguni,2JPLbjOn0wPCngEot2STUS,85,4q3ewBCX7sLwd24euuV69X,2021,97,"[reggaeton, trap latino, urbano latino]",69786971,0.644,...,0.1350,0.440,179.951,audio_features,2JPLbjOn0wPCngEot2STUS,spotify:track:2JPLbjOn0wPCngEot2STUS,https://api.spotify.com/v1/tracks/2JPLbjOn0wPC...,https://api.spotify.com/v1/audio-analysis/2JPL...,206710,4
3,Tainy,Lo Siento BB:/ (with Bad Bunny & Julieta Venegas),4gzsuuZypVbxs0Af1LSZyB,83,0GM7qgcRCORpGnfcN2tCiB,2021,75,"[pop reggaeton, reggaeton, trap latino, urbano...",1358527,0.639,...,0.0865,0.138,169.888,audio_features,4gzsuuZypVbxs0Af1LSZyB,spotify:track:4gzsuuZypVbxs0Af1LSZyB,https://api.spotify.com/v1/tracks/4gzsuuZypVbx...,https://api.spotify.com/v1/audio-analysis/4gzs...,207301,4
4,Rauw Alejandro,Desesperados,6mmPpaltUZK7xjNlBPQQ0p,83,1mcTU81TzQhprhouKaTkpq,2021,89,"[puerto rican pop, reggaeton, trap latino, urb...",16900141,0.869,...,0.0878,0.511,90.020,audio_features,6mmPpaltUZK7xjNlBPQQ0p,spotify:track:6mmPpaltUZK7xjNlBPQQ0p,https://api.spotify.com/v1/tracks/6mmPpaltUZK7...,https://api.spotify.com/v1/audio-analysis/6mmP...,224997,4


In [24]:
track_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name_artist        3000 non-null   object 
 1   name_song          3000 non-null   object 
 2   id_song            3000 non-null   object 
 3   popularity_song    3000 non-null   int64  
 4   id_artist          3000 non-null   object 
 5   year               3000 non-null   object 
 6   popularity_artist  3000 non-null   int64  
 7   genres_artist      3000 non-null   object 
 8   followers_artist   3000 non-null   int64  
 9   danceability       2997 non-null   float64
 10  energy             2997 non-null   float64
 11  key                2997 non-null   object 
 12  loudness           2997 non-null   float64
 13  mode               2997 non-null   object 
 14  speechiness        2997 non-null   float64
 15  acousticness       2997 non-null   float64
 16  instrumentalness   2997 

In [25]:
track_df.agg(["count","nunique"])

C:\Users\JAIRO\AppData\Local\Temp\ipykernel_2256\852557728.py:1: FutureWarning: ['genres_artist'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  track_df.agg(["count","nunique"])


,name_artist,name_song,id_song,popularity_song,id_artist,year,popularity_artist,genres_artist,followers_artist,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
count,3000,3000,3000,3000,3000,3000,3000,3000.0,3000,2997,...,2997,2997,2997,2997,2997,2997,2997,2997,2997,2997
nunique,763,2176,3000,79,765,3,73,NaN,765,564,...,776,829,2188,1,2997,2997,2997,2997,2195,4


In [26]:
track_df = track_df.drop(columns=['key','mode','type', 'id', 'uri','track_href','analysis_url'])

track_df['name_artist'] = track_df['name_artist'].astype("string")
track_df['name_song'] = track_df['name_song'].astype("string")
track_df['id_song'] = track_df['id_song'].astype("string")
track_df['id_artist'] = track_df['id_artist'].astype("string")
track_df['instrumentalness'] = pd.to_numeric(track_df['instrumentalness'])
track_df['duration_ms'] = pd.to_numeric(track_df['duration_ms'])
track_df['time_signature'] = track_df['time_signature'].astype("category")
print(track_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   name_artist        3000 non-null   string  
 1   name_song          3000 non-null   string  
 2   id_song            3000 non-null   string  
 3   popularity_song    3000 non-null   int64   
 4   id_artist          3000 non-null   string  
 5   year               3000 non-null   object  
 6   popularity_artist  3000 non-null   int64   
 7   genres_artist      3000 non-null   object  
 8   followers_artist   3000 non-null   int64   
 9   danceability       2997 non-null   float64 
 10  energy             2997 non-null   float64 
 11  loudness           2997 non-null   float64 
 12  speechiness        2997 non-null   float64 
 13  acousticness       2997 non-null   float64 
 14  instrumentalness   2997 non-null   float64 
 15  liveness           2997 non-null   float64 
 16  valenc

In [27]:
print(track_df.shape)
track_df.head()

(3000, 20)


,name_artist,name_song,id_song,popularity_song,id_artist,year,popularity_artist,genres_artist,followers_artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Yng Lvcas,La Bebe,6odrHMSLjP3aOXC4ipKUC6,86,1NNRWkhwmcXRimFYSBpB1y,2021,81,[],90161,0.775,0.465,-8.365,0.3650,0.6090,0.000032,0.1140,0.707,169.781,223993.0,4
1,Rvssian,Nostálgico,2BEcap3xJtmiPaHKyfQi4e,80,1fctva4kpRbg2k3v7kwRuS,2021,69,"[reggaeton, trap latino, urbano latino]",2387197,0.734,0.846,-3.425,0.0489,0.1660,0.000002,0.2350,0.604,98.026,209884.0,4
2,Bad Bunny,Yonaguni,2JPLbjOn0wPCngEot2STUS,85,4q3ewBCX7sLwd24euuV69X,2021,97,"[reggaeton, trap latino, urbano latino]",69786971,0.644,0.648,-4.601,0.1180,0.2760,0.000000,0.1350,0.440,179.951,206710.0,4
3,Tainy,Lo Siento BB:/ (with Bad Bunny & Julieta Venegas),4gzsuuZypVbxs0Af1LSZyB,83,0GM7qgcRCORpGnfcN2tCiB,2021,75,"[pop reggaeton, reggaeton, trap latino, urbano...",1358527,0.639,0.703,-6.330,0.0794,0.0889,0.000002,0.0865,0.138,169.888,207301.0,4
4,Rauw Alejandro,Desesperados,6mmPpaltUZK7xjNlBPQQ0p,83,1mcTU81TzQhprhouKaTkpq,2021,89,"[puerto rican pop, reggaeton, trap latino, urb...",16900141,0.869,0.694,-3.350,0.0783,0.3560,0.001250,0.0878,0.511,90.020,224997.0,4


In [28]:
df_song = track_df[['id_song', 'name_song', 'popularity_song', 'year', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'id_artist']]
df_song.head()

,id_song,name_song,popularity_song,year,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,id_artist
0,6odrHMSLjP3aOXC4ipKUC6,La Bebe,86,2021,0.775,0.465,-8.365,0.3650,0.6090,0.000032,0.1140,0.707,169.781,223993.0,4,1NNRWkhwmcXRimFYSBpB1y
1,2BEcap3xJtmiPaHKyfQi4e,Nostálgico,80,2021,0.734,0.846,-3.425,0.0489,0.1660,0.000002,0.2350,0.604,98.026,209884.0,4,1fctva4kpRbg2k3v7kwRuS
2,2JPLbjOn0wPCngEot2STUS,Yonaguni,85,2021,0.644,0.648,-4.601,0.1180,0.2760,0.000000,0.1350,0.440,179.951,206710.0,4,4q3ewBCX7sLwd24euuV69X
3,4gzsuuZypVbxs0Af1LSZyB,Lo Siento BB:/ (with Bad Bunny & Julieta Venegas),83,2021,0.639,0.703,-6.330,0.0794,0.0889,0.000002,0.0865,0.138,169.888,207301.0,4,0GM7qgcRCORpGnfcN2tCiB
4,6mmPpaltUZK7xjNlBPQQ0p,Desesperados,83,2021,0.869,0.694,-3.350,0.0783,0.3560,0.001250,0.0878,0.511,90.020,224997.0,4,1mcTU81TzQhprhouKaTkpq


In [29]:
feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

for feature in feature_list:
    df_song[feature] = (df_song[feature]-df_song[feature].min())/(df_song[feature].max() - df_song[feature].min())

df_song.head()

C:\Users\JAIRO\AppData\Local\Temp\ipykernel_2256\1454892428.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_song[feature] = (df_song[feature]-df_song[feature].min())/(df_song[feature].max() - df_song[feature].min())


,id_song,name_song,popularity_song,year,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,id_artist
0,6odrHMSLjP3aOXC4ipKUC6,La Bebe,86,2021,0.738003,0.454378,0.656326,0.417440,0.613896,0.000034,0.106020,0.714225,0.668626,223993.0,4,1NNRWkhwmcXRimFYSBpB1y
1,2BEcap3xJtmiPaHKyfQi4e,Nostálgico,80,2021,0.684825,0.843670,0.832076,0.031387,0.167306,0.000002,0.234949,0.604802,0.259470,209884.0,4,1fctva4kpRbg2k3v7kwRuS
2,2JPLbjOn0wPCngEot2STUS,Yonaguni,85,2021,0.568093,0.641361,0.790238,0.115779,0.278198,0.000000,0.128396,0.430575,0.726617,206710.0,4,4q3ewBCX7sLwd24euuV69X
3,4gzsuuZypVbxs0Af1LSZyB,Lo Siento BB:/ (with Bad Bunny & Julieta Venegas),83,2021,0.561608,0.697558,0.728725,0.068637,0.089582,0.000002,0.076718,0.109742,0.669236,207301.0,4,0GM7qgcRCORpGnfcN2tCiB
4,6mmPpaltUZK7xjNlBPQQ0p,Desesperados,83,2021,0.859922,0.688362,0.834745,0.067294,0.358846,0.001306,0.078103,0.506002,0.213819,224997.0,4,1mcTU81TzQhprhouKaTkpq


In [30]:
df_artist = track_df[['id_artist', 'name_artist', 'followers_artist', 'popularity_artist']]
df_artist = df_artist.drop_duplicates()
print(df_artist.shape)
df_artist.head()

(765, 4)


,id_artist,name_artist,followers_artist,popularity_artist
0,1NNRWkhwmcXRimFYSBpB1y,Yng Lvcas,90161,81
1,1fctva4kpRbg2k3v7kwRuS,Rvssian,2387197,69
2,4q3ewBCX7sLwd24euuV69X,Bad Bunny,69786971,97
3,0GM7qgcRCORpGnfcN2tCiB,Tainy,1358527,75
4,1mcTU81TzQhprhouKaTkpq,Rauw Alejandro,16900141,89


In [31]:
artist_id = track_df[['id_artist', 'genres_artist']]
print(artist_id.shape)
artist_id.head()

(3000, 2)


,id_artist,genres_artist
0,1NNRWkhwmcXRimFYSBpB1y,[]
1,1fctva4kpRbg2k3v7kwRuS,"[reggaeton, trap latino, urbano latino]"
2,4q3ewBCX7sLwd24euuV69X,"[reggaeton, trap latino, urbano latino]"
3,0GM7qgcRCORpGnfcN2tCiB,"[pop reggaeton, reggaeton, trap latino, urbano..."
4,1mcTU81TzQhprhouKaTkpq,"[puerto rican pop, reggaeton, trap latino, urb..."


In [32]:
genres_artist = pd.DataFrame(columns=['id_artist', 'genres_artist'])
genres_artist

,id_artist,genres_artist


In [ ]:
i = 0
for genres in artist_id.genres_artist:
    if len(genres) == 0:
        genres_artist = genres_artist.append({'id_artist': artist_id['id_artist'][i], 'genres_artist': ''}, ignore_index=True)
    else:
        for genre in genres:
            genres_artist = genres_artist.append({'id_artist': artist_id['id_artist'][i], 'genres_artist': genre}, ignore_index=True)
    i = i + 1

genres_artist.head(20)



In [34]:
genres_artist = genres_artist.drop_duplicates()
print(genres_artist.shape)
genres_artist.head(10)

(1585, 2)


,id_artist,genres_artist
0,1NNRWkhwmcXRimFYSBpB1y,
1,1fctva4kpRbg2k3v7kwRuS,reggaeton
2,1fctva4kpRbg2k3v7kwRuS,trap latino
3,1fctva4kpRbg2k3v7kwRuS,urbano latino
4,4q3ewBCX7sLwd24euuV69X,reggaeton
5,4q3ewBCX7sLwd24euuV69X,trap latino
6,4q3ewBCX7sLwd24euuV69X,urbano latino
7,0GM7qgcRCORpGnfcN2tCiB,pop reggaeton
8,0GM7qgcRCORpGnfcN2tCiB,reggaeton
9,0GM7qgcRCORpGnfcN2tCiB,trap latino


In [35]:
with pd.ExcelWriter('data_spotify.xlsx') as writer:  
    df_song.to_excel(writer, sheet_name='Songs', index=False)
    df_artist.to_excel(writer, sheet_name='Artists', index=False)
    genres_artist.to_excel(writer, sheet_name='Genres', index=False)